In [1]:
import glob
import os
import pandas as pd
import re

from commonregex import CommonRegex

from pdfminer.high_level import extract_text
from pdf2image import convert_from_path

from PyPDF2 import PdfReader

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [2]:
# enter the path to a folder where documents are located

path = r'C:/Users/altz7/Downloads/Upwork Docs/Archive/documents'
all_files = glob.glob(path + "/*.pdf")

In [3]:
def pdf_extract():
    file_name_all = []
    phone_all = []
    person_name_all = []
    email_all = []
    company_name_all = []
    address_all = []
    role_all = []
    
    for filename in all_files:
        file_name_all.append(os.path.basename(filename).partition(".")[0])
        
        test_text = extract_text(filename)
        
        #check if PDF is a scan-image or text document
        
        # < 250 - it's a scanned document. Should be processed with Pytesseract
        if len(test_text) < 250:                 
            doc = convert_from_path(filename)
            my_str = ''

            for page_number, page_data in enumerate(doc):
                txt = pytesseract.image_to_string(page_data).encode("utf-8")
                my_str += str(txt)
            
            my_str = my_str.replace("\\n\\n", " ").replace("\\n", " ").replace('"b\'', " ").replace("[]", " ").replace("&", "& ")

            if "DATED:" in my_str:
                raw_text = my_str.partition("DATED:")[2][:350]
            elif "Dated:" in my_str:
                raw_text = my_str.partition("Dated:")[2][:350]
            else:
                raw_text = ""
                        
            raw_text = " ".join(raw_text.split())

            #email
            emailRegex = re.compile(r'''([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+)''', re.VERBOSE)
            email = emailRegex.findall(raw_text)
            if email:
                email_all.append(''.join(email))
            else:
                email_all.append('')
    
            #phone
            phone_raw = re.findall("[(][\d]{3}[)][ ]?[\d]{3}-[\d]{4}", raw_text)
            if phone_raw:
                phone_all.append(''.join(phone_raw))
            else:
                phone_all.append('')

            #address
            address_1 = re.findall('[0-9]{1,3} .+,.+, [A-Z]{2} [0-9]{5}', raw_text)
            address_2 = re.findall('[A-Z]{2,2} [\d]{5,5}', raw_text)
            
            parsed_text = CommonRegex(raw_text)
            address_3 = parsed_text.street_addresses
            
            if address_1:
                address_all.append(''.join(address_1))
            elif address_2:
                address_all.append(''.join(address_2))
            elif address_3:
                address_all.append(address_3)
            else:
                address_all.append("")
                
            # person name
            if "By:" in raw_text and "Attorneys" in raw_text:
                try:
                    name_raw = re.search('By:(.*)Attorneys', raw_text).group(1)
                    name_clean = name_raw.split()
                    name_clean_1 = ' '.join(name_clean[:3])
                    person_name_all.append(name_clean_1)
                except AttributeError:
                    person_name_all.append("")
            else:
                person_name_all.append("")
 
           #role
            if "Attorneys for Plain" in raw_text:
                role_all.append("Attorneys for plaintiff")
            elif "Attorneys for Plaintiff" in raw_text:
                role_all.append("Attorneys for plaintiff")    
            else:
                role_all.append("")

            #company name
            regex_company = r"\b[A-Z]\w+(?:[ -]+(?:&[ -]+)?[A-Z]\w+(?:\.com?)?){0,2}[,\s]+(?i:llp|p.c(?:rp)?|LLP|PLLC|P.C)\b"
            company_name = re.findall(regex_company, raw_text)
            if company_name:
                company_name_all.append(''.join(company_name))
            else:
                company_name_all.append('')

        # > 250 - it's a text document. Should be processed with PdfReader package
        else:
            reader = PdfReader(filename)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            
            text = text.replace("\\n\\n", " ").replace("\\n", " ").replace('"b\'', " ").replace("[]", " ").replace("&", "& ")
            
            if "DATED:" in text:
                raw_text = text.partition("DATED:")[2][:350]
            elif "Dated:" in text:
                raw_text = text.partition("Dated:")[2][:350]
            else:
                raw_text = ""
            
            #email
            emailRegex = re.compile(r'''([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+)''', re.VERBOSE)
            email = emailRegex.findall(raw_text)
            if email:
                email_all.append(''.join(email))
            else:
                email_all.append('')
    
            #phone
            phone_raw = re.findall("[(][\d]{3}[)][ ]?[\d]{3}-[\d]{4}", raw_text)
            if phone_raw:
                phone_all.append(''.join(phone_raw))
            else:
                phone_all.append('')

            #address
            address_1 = re.findall('[0-9]{1,3} .+,.+, [A-Z]{2} [0-9]{5}', raw_text)
            address_2 = re.findall('[A-Z]{2,2} [\d]{5,5}', raw_text)
            
            parsed_text = CommonRegex(raw_text)
            address_3 = parsed_text.street_addresses
            
            if address_1:
                address_all.append(''.join(address_1))
            elif address_2:
                address_all.append(''.join(address_2))
            elif address_3:
                address_all.append(address_3)
            else:
                address_all.append("")
            
            # person name
            if "By:" in raw_text and "Attorneys" in raw_text:
                try:
                    name_raw = re.search('By:(.*)Attorneys', raw_text).group(1)
                    name_clean = name_raw.split()
                    name_clean_1 = ' '.join(name_clean[:3])
                    person_name_all.append(name_clean_1)
                except AttributeError:
                    person_name_all.append("")
            else:
                person_name_all.append("")
   
            #role
            if "Attorneys for Plain" in raw_text:
                role_all.append("Attorneys for plaintiff")
            elif "Attorneys for Plaintiff" in raw_text:
                role_all.append("Attorneys for plaintiff")    
            else:
                role_all.append("")

            #company name
            regex_company = r"\b[A-Z]\w+(?:[ -]+(?:&[ -]+)?[A-Z]\w+(?:\.com?)?){0,2}[,\s]+(?i:llp|p.c(?:rp)?|LLP|PLLC|P.C)\b"
            company_name = re.findall(regex_company, raw_text)
            if company_name:
                company_name_all.append(''.join(company_name))
            else:
                company_name_all.append('')
            
    df = pd.DataFrame(data=zip(file_name_all,
                               person_name_all,
                               company_name_all,
                               address_all,
                               phone_all,
                               email_all,
                               role_all),
                       columns=['file_name', 'person_name', 'law_firm', 'address', 'phone', 'email', 'role'])
    
    return df